In [338]:
import pandas as pd
from AUIDFunctions import *
uh = pd.read_csv("input/unit-history_demo.csv")
am = pd.read_csv("input/all-members_demo.csv")
ac = pd.read_csv("input/accused_demo.csv")
am = am.rename(columns = {'TID' : 'ID1'})
uh = uh.rename(columns = {'TID' : 'ID2'})

In [339]:
def intersect(a, b):
     return list(set(a) & set(b))
def setdiff(a,b):
    return list(set(a) - set(b))
def union(a,b):
    return list(set(a) | set(b))

In [340]:
TakeFirst4 = lambda x: x[:4]
BY_to_CA = lambda x: 2016 - x
def AddColumns(df, add_cols = ["F4FN", "F4LN", "Current.Age", "CA_BY"]):
    if "F4FN" in add_cols and "First.Name" in df.columns:
            df['F4FN'] = df['First.Name'].map(TakeFirst4)
    if "F4LN" in add_cols and 'Last.Name' in df.columns:
            df['F4LN'] = df['Last.Name'].map(TakeFirst4)
    if "Current.Age" in add_cols and "Current.Age" in df.columns:
        df['Current.Age.p1'] = df['Current.Age']
        df['Current.Age.m1'] = df['Current.Age']
    if "CA_BY" in add_cols and "Birth.Year" in df.columns:
        by_to_ca = lambda x: 2016 - x
        df['Current.Age.p1'] = df['Birth.Year'].map(BY_to_CA)
        df['Current.Age.m1'] = df['Birth.Year'].map(BY_to_CA) - 1  
    return df



In [341]:
amd = am.dropna(axis=1, how='all')
uhd = uh.dropna(axis=1, how='all')
amd = AddColumns(amd)
uhd = AddColumns(uhd)
print(amd.columns)
print(uhd.columns)
cols = [col for col in amd.columns if col in uhd.columns]
print(cols)

Index(['ID1', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'F4FN', 'F4LN',
       'Current.Age.p1', 'Current.Age.m1'],
      dtype='object')
Index(['ID2', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Current.Age', 'Gender', 'Race', 'Star1', 'Star2',
       'Star3', 'Star4', 'Star5', 'Star6', 'Star7', 'Star8', 'Star9', 'Star10',
       'Current.Unit', 'F4FN', 'F4LN', 'Current.Age.p1', 'Current.Age.m1'],
      dtype='object')
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'F4FN', 'F4LN', 'Current.Age.p1', 'Current.Age.m1']


In [342]:
mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t1 = amdmp.merge(uhdmp, on = mc, how ='inner')

mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t2 = amdmp.merge(uhdmp, on = mc, how ='inner')

print(intersect(t1['ID2'], t2['ID2']))
print(intersect(t1['ID1'], t2['ID1']))
print(t1.shape)
print(t2.shape)

[]
[]
(1312, 16)
(4289, 16)


In [343]:
def LoopMerge(df1, df2, on_list, keep_columns = ['ID1', 'ID2']):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    # Generate on list?
    for mc in on_list:
        df1t = RemoveDuplicates(df1[keep_columns[:1] + mc], mc)
        df2t = RemoveDuplicates(df2[keep_columns[1:] + mc], mc)
        dfmt = df1t.merge(df2t, on=mc, how='inner')
        if dfmt.shape[0] > 0:
            print('******')
            print(mc)
            print(dfmt.shape[0])
            dfmt['Match'] = '-'.join(mc)
            dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
            df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
            df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]

    print(df1.shape[0])
    print(df2.shape[0])
    return (dfm.reset_index(drop=True), df1, df2)



In [344]:
m = LoopMerge(amdm, uhdm, on_list = [
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"]
])

******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
4289
******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.p1']
1312
******
['F4FN', 'F4LN', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
1
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
19949
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.p1']
6011
******
['F4FN', 'F4LN', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
3
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Middle.Initial', 'Appointed.Date', 'Gender', 'Race']
1
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race']
2
******
['First.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
4
******
['First

In [345]:
def AppendToReference(ref, df):
    return 0

In [346]:
uid_ref = pd.concat([m[0][["ID1", "ID2"]], m[1][["ID1"]], m[2][["ID2"]]]).reset_index(drop=True)
if "UID" not in uid_ref.columns:
    uid_ref.insert(0, "UID", uid_ref.index + 1)

In [347]:
am = am.merge(uid_ref[["ID1", "UID"]], on='ID1', how='left')
uh = uh.merge(uid_ref[["ID2", "UID"]], on='ID2', how='left')

In [348]:
ref = pd.concat([am, uh])

In [349]:
profiles = AggregateData(ref, uid = "UID", id_cols = [],
              max_cols = setdiff(ref.columns, ["UID"]))

In [350]:
profiles

UID  Star10    Star1  Current.Age  Birth.Year  Current.Unit  \
0       5603  -999.0  13001.0         44.0      1971.0          14.0   
1          1  -999.0  17545.0         35.0      1980.0          15.0   
2      25552  -999.0  14535.0         74.0      1942.0         543.0   
3       5604  -999.0  18601.0         47.0      1968.0          20.0   
4       5605  -999.0  20484.0         73.0      1942.0         640.0   
5       5606  -999.0   -999.0         85.0      1930.0          25.0   
6      25553  -999.0  19596.0         44.0      1972.0          16.0   
7       4290  -999.0   -999.0         93.0      1923.0         132.0   
8       5607  -999.0   6799.0         70.0      1945.0         192.0   
9       5608  -999.0  16511.0         63.0      1952.0         151.0   
10     25554  -999.0  20921.0         67.0      1949.0         606.0   
11     25555  -999.0  11969.0         63.0      1953.0          50.0   
12      5609  -999.0   4670.0         57.0      1958.0          50.0   
13      5610  -999.0  21167.0         50.0      1965.0         630.0   
14      5611  -999.0  17673.0         32.0      1983.0           6.0   
15         2  -999.0   7147.0         37.0      1978.0         191.0   
16         3  -999.0  14048.0         31.0      1984.0           1.0   
17      5612  -999.0   4527.0         57.0      1958.0          19.0   
18     25556  -999.0  14237.0         40.0      1976.0          19.0   
19      5613  -999.0   5959.0         66.0      1949.0          17.0   
20     25557  -999.0  14171.0         67.0      1949.0         640.0   
21      5614  -999.0  17436.0         58.0      1957.0           4.0   
22      5615  -999.0   3473.0         78.0      1937.0         142.0   
23      5616  -999.0  11360.0         25.0      1990.0          44.0   
24      4291  -999.0   -999.0         91.0      1925.0          16.0   
25      5617  -999.0  11110.0         93.0      1922.0           8.0   
26      5618  -999.0  16579.0         29.0      1986.0          11.0   
27     31583     NaN      NaN          NaN      1988.0           NaN   
28      5619  -999.0   1109.0         76.0      1939.0         701.0   
29      5620  -999.0  11485.0         52.0      1963.0          50.0   
...      ...     ...      ...          ...         ...           ...   
31913  31561  -999.0  10336.0         65.0      1951.0          71.0   
31914  31562  -999.0  16055.0         37.0      1979.0           1.0   
31915  25542  -999.0  15918.0         48.0      1967.0          19.0   
31916  25543  -999.0   -999.0         99.0      1916.0           NaN   
31917  25544  -999.0  14383.0         82.0      1933.0         151.0   
31918  25545  -999.0  13312.0         25.0      1990.0          44.0   
31919  25546  -999.0   5642.0         31.0      1984.0           7.0   
31920  31927     NaN      NaN          NaN      1990.0           NaN   
31921   4289  -999.0  12477.0         74.0      1941.0         188.0   
31922  25547  -999.0  19965.0         45.0      1970.0          19.0   
31923  31928     NaN      NaN          NaN      1980.0           NaN   
31924  25548  -999.0  21324.0         59.0      1956.0         650.0   
31925  25549  -999.0   6568.0         41.0      1974.0          14.0   
31926  25550  -999.0  19215.0         45.0      1970.0          20.0   
31927  25551  -999.0   2845.0         83.0      1932.0         631.0   
31928  31929  -999.0   8024.0         67.0         NaN         128.0   
31929  31930  -999.0  14297.0         68.0         NaN          16.0   
31930  31931  -999.0  10927.0         32.0         NaN           5.0   
31931  31932  -999.0   8791.0         33.0         NaN           4.0   
31932  31933  -999.0   2010.0         90.0         NaN         165.0   
31933  31934  -999.0   -999.0         89.0         NaN          11.0   
31934  31935  -999.0   8997.0         58.0         NaN         171.0   
31935  31936  -999.0  14337.0         57.0         NaN           3.0   
31936  31937  -999.0  40002.0         68.0       

In [351]:
ac.shape

(9027, 12)